# PyCity Schools Analysis

# Results

* In general, the students performed better in Reading that in Math

* The Pena High School was the school with the overall best results 

* The school with the lowest overall grade was the Rodriguez High School

* The average scores per school year were similar

* A higher budget does not imply a higher score. On the other hand, the smaller the group the higher the score.

* All the charter schools had a better score rate than the district schools.



In [405]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from copy import copy

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [406]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


## General description of the schools data 
The summary includes the following metrics:
* The total number of schools

* The total number of students

* The total budget

* The average math score 

* The average reading score

* The overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* The percentage of students with a passing math score (70 or greater)

* The percentage of students with a passing reading score (70 or greater)

* A dataframe that holds the above results is displayed

In [407]:
#The first step is check if there is missing data, duplicateds or fields with the wrong format
school_data_complete.dtypes
school_data_complete.shape
school_data_complete.count()
duplicates= pd.DataFrame({"Boolean":school_data_complete.duplicated()})
duplicates2 = duplicates.loc[duplicates["Boolean"] == True,:]
len(duplicates2)

0

In [408]:
#As everything is ok we proceed with the operations
number_of_sutdentsID = len(school_data_complete["Student ID"].unique())
number_of_schools = len(school_data_complete["school_name"].unique())
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()

#This passing rate was calculated using the average mat score and average reading score of all students
average_passing_rate = (average_math_score + average_reading_score)/2

In [409]:
#The total budget was calculated with group by to avoid school budget duplicates
grouped_by_school = school_data_complete.groupby(['school_name'])
budget_average = grouped_by_school.mean()
total_budget=budget_average["budget"].sum()

In [410]:
#Now we get the % of students with a passing math score and reading score > 70
passing_math_perc = ((len(school_data_complete.loc[school_data_complete["math_score"] >= 70,:])) / number_of_sutdentsID) * 100
passing_reading_perc = ((len(school_data_complete.loc[school_data_complete["reading_score"] >= 70,:])) / number_of_sutdentsID) * 100

In [411]:
#A data frame with all the previous information is created
summary_table = pd.DataFrame({"Total Schools": number_of_schools,
                                    "Total Students": number_of_sutdentsID,
                                    "Total Budget": total_budget,
                                    "Average Math Score": average_math_score,
                                    "Average Reading Score": average_reading_score,
                                    "% Passing Math": [passing_math_perc] ,
                                    "% Passing Reading": [passing_reading_perc],
                                    "Overall Passing Rate": average_passing_rate
                                   },  columns=["Total Schools", "Total Students","Total Budget","Average Math Score",\
                                                "Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"])
summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,80.431606


In [412]:
#A map function is used to format the values
summary_table["Total Budget"] = summary_table["Total Budget"].map("${:,}".format) 
summary_table["Total Students"] = summary_table["Total Students"].map("{:,}".format) 
summary_table["Average Math Score"] = summary_table["Average Math Score"].map("{:.2f}".format) 
summary_table["Average Reading Score"] = summary_table["Average Reading Score"].map("{:.2f}".format)
summary_table["% Passing Math"] = summary_table["% Passing Math"].map("{:.2f}".format)
summary_table["% Passing Reading"] = summary_table["% Passing Reading"].map("{:.2f}".format)
summary_table["Overall Passing Rate"] = summary_table["Overall Passing Rate"].map("{:.2f}".format)
summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.0",78.99,81.88,74.98,85.81,80.43


## School Summary

*In this part an overview table that summarizes key metrics about each school is displayed:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  

## Top Performing Schools (By Passing Rate)

In [413]:
#The next step is to get the metrics for each school

#First the metrics that are calculated with the mean fucntion: average reading score and math score
school_group = school_data_complete.groupby(['school_name'])
school_group_scores=school_group["reading_score","math_score"]
school_group_scores=school_group_scores.mean()

In [414]:
#Then the metrics that required a conditional function were calculated: Scores of students above or equal to 70

#Sellect by conditional
passing_math_conditional = (school_data_complete.loc[school_data_complete["math_score"] >= 70,:])
passing_reading_conditional = school_data_complete.loc[school_data_complete["reading_score"] >= 70,:]

#Extract the scores per school using groupby
passing_math_per_school= passing_math_conditional .groupby(['school_name'])["math_score"]
passing_reading_per_school =passing_reading_conditional.groupby(['school_name'])["reading_score"]

#Calculate the mean of the students who passed math and reading
passing_math_mean= passing_math_per_school.mean()
passing_reading_mean= passing_reading_per_school.mean()

#Transform the Series into a DataFrame
passing_math_mean=pd.DataFrame({'School':passing_math_mean.index, 'Passing math score':passing_math_mean.values})
passing_reading_mean=pd.DataFrame({'School':passing_reading_mean.index, 'Passing reading score':passing_reading_mean.values})

#Merge both DataFrames and make the school tame the index of the dataframe
passing_grade = pd.merge(passing_math_mean, passing_reading_mean, on= 'School')
passing_grade = passing_grade.set_index('School')

In [415]:
#The total students per scool is not a parameter given in the original tables so it is neccesary to calculate it 
#counting the number of rows per school
total_students = pd.DataFrame({"Total Students":school_data_complete["school_name"].value_counts()})

In [416]:
#The school type and total budget are parameters given in the orighinal data but those are needed without duplicates

#Extract from the original data the school name, the type and the budget
types= pd.DataFrame({"School name":school_data_complete["school_name"],"School type":school_data_complete["type"],"Total Budget":school_data_complete["budget"]})

#Drop duplicateds and make the school name the index of the dataframe
types.drop_duplicates(keep="first",inplace=True) 
types = types.set_index("School name")

In [417]:
#Merge current data by index-School Name
merge_table = pd.merge(types, total_students, left_index=True, right_index=True )
merge_table = pd.merge(merge_table, school_group_scores, left_index=True, right_index=True )
merge_table = pd.merge(passing_grade, merge_table, left_index=True, right_index=True )

In [418]:
#The budget per students, and the overall passing grade still need to be calculated using the current data
merge_table["Budget per Student"]=merge_table["Total Budget"]/merge_table["Total Students"]

#The overall passing rate considers all students
merge_table["Overall Passing Rate"]= (merge_table["reading_score"] + merge_table["math_score"])/2


In [419]:
#Then the columns are sorted in the desired order 
merge_table = merge_table[["School type","Total Students","Total Budget","Budget per Student","math_score",\
                           "reading_score","Passing math score","Passing reading score","Overall Passing Rate"]]

In [420]:
#A more appropiate name are given to some metrics
merge_table = merge_table.rename(columns={"math_score":"Average Math Score",\
                                          "reading_score":"Average Reading Score",\
                                          "Passing math score":"Passing Math score > 70",\
                                          "Passing reading score":"Passing Reading score > 70"})
final_table = copy(merge_table)

In [421]:
#Change the format for the budgets
final_table["Total Budget"] = final_table["Total Budget"].map("${:,}".format) 
final_table["Budget per Student"] = final_table["Budget per Student"].map("${:,}".format) 
final_table["Total Students"] = final_table["Total Students"].map("{:,}".format) 
final_table["Average Math Score"] = final_table["Average Math Score"].map("{:.2f}".format) 
final_table["Average Reading Score"] = final_table["Average Reading Score"].map("{:.2f}".format)
final_table["Passing Math score > 70"] = final_table["Passing Math score > 70"].map("{:.2f}".format)
final_table["Passing Reading score > 70"] = final_table["Passing Reading score > 70"].map("{:.2f}".format)
final_table["Overall Passing Rate"] = final_table["Overall Passing Rate"].map("{:.2f}".format)

In [428]:
#Get the top performing schools
top_performing = final_table.sort_values("Overall Passing Rate", ascending=False)
top_performing

,School type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Passing Math score > 70,Passing Reading score > 70,Overall Passing Rate
Pena High School,Charter,962,"$585,858",$609.0,83.84,84.04,84.72,84.68,83.94
Wright High School,Charter,"1,800","$1,049,400",$583.0,83.68,83.95,84.76,84.48,83.82
Holden High School,Charter,427,"$248,087",$581.0,83.80,83.81,85.04,84.39,83.81
Thomas High School,Charter,"1,635","$1,043,130",$638.0,83.42,83.85,84.50,84.26,83.63
Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.27,83.99,84.24,84.53,83.63
Griffin High School,Charter,"1,468","$917,500",$625.0,83.35,83.82,84.39,84.25,83.58
Shelton High School,Charter,"1,761","$1,056,600",$600.0,83.36,83.73,84.33,84.36,83.54
Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06,83.98,83.97,84.43,83.52
Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29,80.93,84.94,84.48,79.11
Bailey High School,District,"4,976","$3,124,928",$628.0,77.05,81.03,84.51,84.36,79.04


## Bottom Performing Schools (By Passing Rate)

In [426]:
#Get the bottom performing schools
bottom_performing = final_table.sort_values("Overall Passing Rate", ascending=True)
bottom_performing

,School type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Passing Math score > 70,Passing Reading score > 70,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.84,80.74,84.34,84.37,78.79
Huang High School,District,"2,917","$1,910,635",$655.0,76.63,81.18,84.24,84.69,78.91
Ford High School,District,"2,739","$1,763,916",$644.0,77.10,80.75,84.17,84.61,78.92
Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71,81.16,84.31,84.77,78.93
Johnson High School,District,"4,761","$3,094,650",$650.0,77.07,80.97,84.74,84.43,79.02
Bailey High School,District,"4,976","$3,124,928",$628.0,77.05,81.03,84.51,84.36,79.04
Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29,80.93,84.94,84.48,79.11
Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06,83.98,83.97,84.43,83.52
Shelton High School,Charter,"1,761","$1,056,600",$600.0,83.36,83.73,84.33,84.36,83.54
Griffin High School,Charter,"1,468","$917,500",$625.0,83.35,83.82,84.39,84.25,83.58


## Math Scores by Grade

*In this part a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school is created.


In [385]:
#grouped_school = school_data_complete.groupby(['school_name'])
grade_9th =(school_data_complete.loc[school_data_complete["grade"] == "9th",["school_name","math_score"]]).set_index("school_name")
grade_10th =(school_data_complete.loc[school_data_complete["grade"] == "10th",["school_name","math_score"]]).set_index("school_name")
grade_11th =(school_data_complete.loc[school_data_complete["grade"] == "11th",["school_name","math_score"]]).set_index("school_name")
grade_12th =(school_data_complete.loc[school_data_complete["grade"] == "12th",["school_name","math_score"]]).set_index("school_name")

#group each series by school
grouped_grade_9th = (grade_9th.groupby(['school_name'])).mean()
grouped_grade_10th = (grade_10th.groupby(['school_name'])).mean()
grouped_grade_11th = (grade_11th.groupby(['school_name'])).mean()
grouped_grade_12th = (grade_12th.groupby(['school_name'])).mean()   

In [386]:
#group each series by school
grouped_math_grade = grouped_grade_9th.merge(grouped_grade_10th, left_index=True, right_index=True, suffixes=["_9th","_10th"])\
                    .merge(grouped_grade_11th, left_index=True, right_index=True, suffixes=["_10th","_11th"])\
                    .merge(grouped_grade_12th, left_index=True, right_index=True, suffixes=["_11th","_12th"])
grouped_math_grade

,math_score_9th,math_score_10th,math_score_11th,math_score_12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [425]:
math_grade_summary = grouped_math_grade.mean()
math_grade_summary

math_score_9th     80.351617
math_score_10th    80.378948
math_score_11th    80.575873
math_score_12th    80.423811
dtype: float64

## Reading Score by Grade 

In [387]:
#grouped_school = school_data_complete.groupby(['school_name'])
grade_9th_r =(school_data_complete.loc[school_data_complete["grade"] == "9th",["school_name","reading_score"]]).set_index("school_name")
grade_10th_r =(school_data_complete.loc[school_data_complete["grade"] == "10th",["school_name","reading_score"]]).set_index("school_name")
grade_11th_r =(school_data_complete.loc[school_data_complete["grade"] == "11th",["school_name","reading_score"]]).set_index("school_name")
grade_12th_r =(school_data_complete.loc[school_data_complete["grade"] == "12th",["school_name","reading_score"]]).set_index("school_name")

#group each series by school
grouped_grade_9th_r = (grade_9th_r.groupby(['school_name'])).mean()
grouped_grade_10th_r = (grade_10th_r.groupby(['school_name'])).mean()
grouped_grade_11th_r = (grade_11th_r.groupby(['school_name'])).mean()
grouped_grade_12th_r = (grade_12th_r.groupby(['school_name'])).mean()

In [388]:
#Merge all data into a single DataFrame
grouped_reading_grade = grouped_grade_9th_r.merge(grouped_grade_10th_r, left_index=True, right_index=True, suffixes=["_9th","_10th"])\
                    .merge(grouped_grade_11th_r, left_index=True, right_index=True, suffixes=["_10th","_11th"])\
                    .merge(grouped_grade_12th_r, left_index=True, right_index=True, suffixes=["_11th","_12th"])
grouped_reading_grade

,reading_score_9th,reading_score_10th,reading_score_11th,reading_score_12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [423]:
reading_grade_summary =grouped_reading_grade.mean()
reading_grade_summary

reading_score_9th     82.513318
reading_score_10th    82.505439
reading_score_11th    82.559485
reading_score_12th    82.554817
dtype: float64

## Scores by School Spending

* In this part a table that breaks down school performances based on average Spending Ranges (Per Student) is created.  The table includes the following metrics:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [389]:
# Create the bins for the metric budget per student
spending_bins = [0, 585, 615, 645, 675]
group_names_spendings = ["<$585", "$585-615", "$615-645", "$645-675"]

In [395]:
# Use the table with the metrics by school as reference and perform the cut function
#and add a new column with the new data
spendings = merge_table
spendings["Spending Groups"] = pd.cut(spendings["Budget per Student"], spending_bins, labels = group_names_spendings)

In [392]:
# Group the dataframe by bins
spending_summary = (spendings.groupby(['Spending Groups']))["Average Math Score","Average Reading Score",\
                                    "Passing Math score > 70", "Passing Reading score > 70","Overall Passing Rate"]
spending_summary = spending_summary.mean()
spending_summary

,Average Math Score,Average Reading Score,Passing Math score > 70,Passing Reading score > 70,Overall Passing Rate
Spending Groups,,,,,
$585-615,83.599686,83.885211,84.523229,84.521475,83.742449
$615-645,79.079225,81.891436,84.368854,84.438364,80.485330
$645-675,76.997210,81.027843,84.639836,84.535230,79.012526
<$585,83.455399,83.933814,84.504010,84.457674,83.694607


## Scores by School Size

In [396]:
# Create the bins for the metric total students and repeat the operations as above
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [400]:
school_size = merge_table
school_size["Size Groups"] = pd.cut(school_size["Total Students"], size_bins, labels = size_group_names)

In [399]:
size_summary = (school_size.groupby(['Size Groups']))["Average Math Score","Average Reading Score",\
                                    "Passing Math score > 70", "Passing Reading score > 70","Overall Passing Rate"]
size_summary = size_summary.mean()
size_summary

,Average Math Score,Average Reading Score,Passing Math score > 70,Passing Reading score > 70,Overall Passing Rate
Size Groups,,,,,
Large (2000-5000),77.746417,81.344493,84.435547,84.531238,79.545455
Medium (1000-2000),83.374684,83.864438,84.390094,84.357500,83.619561
Small (<1000),83.821598,83.929843,84.880143,84.536059,83.875721


## Scores by School Type

In [403]:
# Use the table with the metrics by school as reference and group by school type
type_scores = merge_table.groupby(["School type"])["Average Math Score","Average Reading Score",\
                                    "Passing Math score > 70", "Passing Reading score > 70","Overall Passing Rate"]

# Compute the mean
type_scores = type_scores.mean()
type_scores

,Average Math Score,Average Reading Score,Passing Math score > 70,Passing Reading score > 70,Overall Passing Rate
School type,,,,,
Charter,83.473852,83.896421,84.494351,84.423298,83.685136
District,76.956733,80.966636,84.462903,84.531876,78.961685
